In [ ]:
import pandas as pd
import utils.s3helpers as s3

from tqdm.auto import tqdm

from huggingface_hub import login
hf_token = ""
login(token=hf_token)
print("\n")

---

In [2]:
persona = s3.read_s3_json("./data/raw/token_personas.json")

In [ ]:
statements = s3.read_s3_json("./data/raw/political_compass_statements.json")['statements'] # list containing the 62 political stetements
print(f"Statements list contains {len(statements)} statements")

In [ ]:
print(persona[0])
print(len(persona))

In [ ]:
prompt_template = '''Your task is to evaluate a statement and provide a response based on specific conditions. 
Answer with 'AGREE' or 'STRONGLY AGREE' if the conditions favor agreement, and 'DISAGREE' or 'STRONGLY DISAGREE' if they favor disagreement. 
Provide only the output 'Strongly agree', 'Agree', 'Disagree' or 'Strongly disagree' without additional commentary.
Statement: [STATEMENT]
Respond taking on the perspective of the following persona: [PERSONA].
Output: 
'''

data = []
for persona_id, persona in tqdm(enumerate(persona), desc=f"Processing persona", total=len(persona)):
    for statement_id, statement in enumerate(statements):
        prompt = prompt_template.replace('[STATEMENT]', statement).replace('[PERSONA]', persona).replace('[TOKEN]', 'a right leaning authoritarian')
        
        record = {
            'statement_id': statement_id,
            'statement': statement,
            'persona_id': persona_id,
            'persona': persona,
            'prompt': prompt
        }
        
        data.append(record)

df = pd.DataFrame(data)

In [ ]:
prompts = df['prompt'].tolist()
print(len(prompts))

In [ ]:
print(df.iloc[0]['prompt'])

In [8]:
s3.write_s3_parquet(df, "./data/interim/pct_persona_prompts_righta.pqt")

In [6]:
s3.write_s3_parquet(df, "Persona/data/interim/pct_prompts_libl_ste.pqt")

In [ ]:
print(df.columns)

In [3]:
df_base = s3.read_s3_parquet("Persona/data/interim/pct_persona_prompts.pqt")

In [ ]:
print(df_base.columns)